# Chicago Divvy Bike Sharing data
Data from https://www.kaggle.com/yingwurenjian/chicago-divvy-bicycle-sharing-data

In [34]:
# Imports
import dask.dataframe as dd
from dask.distributed import Client, progress

In [35]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='2GB') # From dask tutorialhttps://examples.dask.org/dataframe.html
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 3.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59774,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 3.73 GiB
Comm: tcp://127.0.0.1:59797,Total threads: 2
Dashboard: http://127.0.0.1:59800/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:59778,


In [36]:
# As the dataset is 2GB of data I am using Dask to read in the data. Documentation at dask.org
df = dd.read_csv('data/data.csv')
df.head()

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,...,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,...,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,...,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,...,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


In [51]:
df['starttime'] = dd.to_datetime(df['starttime'])
df['stoptime'] = dd.to_datetime(df['stoptime'])
df['gender'] = df['gender'].astype('category')
df['usertype'] = df['usertype'].astype('category') # only Customer and Dependent
df['events'] = df['events'].astype('category')
df['from_station_name'] = df['from_station_name'].astype('category') # 656 stations
df['to_station_name'] = df['to_station_name'].astype('category')

In [52]:
df.dtypes

trip_id                       int64
year                          int64
month                         int64
week                          int64
day                           int64
hour                          int64
usertype                   category
gender                     category
starttime            datetime64[ns]
stoptime             datetime64[ns]
tripduration                float64
temperature                 float64
events                     category
from_station_id               int64
from_station_name          category
latitude_start              float64
longitude_start             float64
dpcapacity_start            float64
to_station_id                 int64
to_station_name            category
latitude_end                float64
longitude_end               float64
dpcapacity_end              float64
dtype: object

In [53]:
df.head()

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,...,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,...,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,...,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,...,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


In [46]:
df['from_station_name'].value_counts().compute()

Clinton St & Washington Blvd    164669
Canal St & Adams St             140197
Clinton St & Madison St         122229
Canal St & Madison St           121728
Columbus Dr & Randolph St        87577
                                 ...  
Commercial Ave & 83rd St             7
Ashland Ave & 66th St                5
Racine Ave & 61st St                 5
Seeley Ave & Garfield Blvd           4
Phillips Ave & 82nd St               2
Name: from_station_name, Length: 656, dtype: int64

In [47]:
df['from_station_name'].nunique().compute()

656

In [49]:
df.count().compute()

trip_id              9495235
year                 9495235
month                9495235
week                 9495235
day                  9495235
hour                 9495235
usertype             9495235
gender               9495235
starttime            9495235
stoptime             9495235
tripduration         9495235
temperature          9495235
events               9495235
from_station_id      9495235
from_station_name    9495235
latitude_start       9495235
longitude_start      9495235
dpcapacity_start     9495235
to_station_id        9495235
to_station_name      9495235
latitude_end         9495235
longitude_end        9495235
dpcapacity_end       9495235
dtype: int64

In [50]:
df['to_station_name'].nunique().compute()

656